In [ ]:
import os
import torch
from torch import nn, optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models


In [12]:
import os
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

# Paths
data_dir = "C:/Users/Public/Dev/DeepLearning/umam/dataset"  # Path to the dataset folder

# Parameters
IMG_SIZE = 224
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2  # Use 20% of data for validation

# Data Preprocessing
data_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the dataset
full_dataset = datasets.ImageFolder(data_dir, transform=data_transforms)

# Split the dataset into train and validation sets
total_size = len(full_dataset)
val_size = int(total_size * VALIDATION_SPLIT)
train_size = total_size - val_size

train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [13]:
# Device configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained ResNet18
model = models.resnet18(pretrained=True)

# Modify the final layer to match the number of classes (15)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 15)  # 15 classes in your dataset

model = model.to(DEVICE)


c:\Users\shash\anaconda3\envs\torch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\shash\anaconda3\envs\torch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\shash/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:12<00:00, 3.89MB/s]


In [14]:
# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [15]:
# Training Parameters
EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    # Training loop
    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)

        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Update statistics
        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    # Calculate training accuracy
    train_accuracy = correct / total
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {running_loss:.4f}, Accuracy: {train_accuracy:.4f}")


Epoch 1/10, Loss: 53.5414, Accuracy: 0.6652
Epoch 2/10, Loss: 29.3946, Accuracy: 0.8181
Epoch 3/10, Loss: 18.9693, Accuracy: 0.8702
Epoch 4/10, Loss: 14.8624, Accuracy: 0.9042
Epoch 5/10, Loss: 11.2704, Accuracy: 0.9344
Epoch 6/10, Loss: 12.5803, Accuracy: 0.9184
Epoch 7/10, Loss: 11.2157, Accuracy: 0.9248
Epoch 8/10, Loss: 9.3746, Accuracy: 0.9441
Epoch 9/10, Loss: 5.6807, Accuracy: 0.9698
Epoch 10/10, Loss: 5.9287, Accuracy: 0.9634


In [17]:
# Validation Loop
model.eval()  # Set model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # Disable gradient calculation for validation
    for images, labels in val_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        # Forward pass
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        # Update statistics
        correct += (preds == labels).sum().item()
        total += labels.size(0)

val_accuracy = correct / total
print(f"Validation Accuracy: {val_accuracy:.4f}")


Validation Accuracy: 0.8840


In [18]:
torch.save(model.state_dict(), "animal_classifier.pth")
print("Model saved!")


Model saved!


In [20]:
# Load the saved model
model.load_state_dict(torch.load("animal_classifier.pth"))
model.eval()

# Test with a single image
from PIL import Image

image_path = "dataset\Bird\Bird_2_2.jpg"
image = Image.open(image_path)
image = data_transforms(image).unsqueeze(0).to(DEVICE)

with torch.no_grad():
    output = model(image)
    _, predicted_class = torch.max(output, 1)

# Map the prediction to class names
class_names = full_dataset.classes
print(f"Predicted class: {class_names[predicted_class.item()]}")


Predicted class: Bird


C:\Users\shash\AppData\Local\Temp\ipykernel_33432\2287365566.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("animal_classifier.pth"))
